In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse
import json
import re
import numpy as np
import pymorphy2
import gc
import itertools
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from datetime import datetime
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.corpus import words
from razdel import tokenize
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from langdetect import detect
from langdetect import detect_langs
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from gensim.test.utils import datapath

### Обработка данных:

In [4]:
#источник данных: https://www.kaggle.com/austinreese/goodreads-books
books = pd.read_csv('goodreads_books.csv', quotechar='\"', 
                 escapechar='\\', 
                 error_bad_lines=False, nrows=20000)

#books.columns

In [5]:
books.head(3)

,id,title,link,series,cover_link,author,author_link,rating_count,review_count,average_rating,...,isbn13,asin,settings,characters,awards,amazon_redirect_link,worldcat_redirect_link,recommended_books,books_in_series,description
0,630104,Inner Circle,https://www.goodreads.com//book/show/630104.In...,(Private #5),https://i.gr-assets.com/images/S/compressed.ph...,"Kate Brian, Julian Peploe",https://www.goodreads.com/author/show/94091.Ka...,7597,196,4.03,...,9781416950417,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"726458, 726458, 1537534, 3047848, 1651302, 304...","381489, 381501, 352428, 630103, 1783281, 17832...",Reed Brennan arrived at Easton Academy expecti...
1,9487,A Time to Embrace,https://www.goodreads.com//book/show/9487.A_Ti...,(Timeless Love #2),https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,https://www.goodreads.com/author/show/3159984....,4179,177,4.35,...,9781595542328,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"127352, 127352, 40642197, 127353, 127354, 3891...",115036,"Ideje az Ã¶lelÃ©snek TÃ¶rtÃ©net a remÃ©nyrÅl,..."
2,6050894,Take Two,https://www.goodreads.com//book/show/6050894-t...,(Above the Line #2),https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,https://www.goodreads.com/author/show/3159984....,6288,218,4.23,...,NaN,NaN,"Bloomington, Indiana(United States)",NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"706250, 706250, 666481, 11942636, 706241, 1273...","4010795, 40792877, 7306261",Filmmakers Chase Ryan and Keith Ellison have c...


In [6]:
books = books[~books['description'].isnull()][['genre_and_votes', 'description']]
books['book_id'] = range(len(books))
books.index = range(len(books))
#books.head(3)
len(books)

19019

In [7]:
#удаляем все книги с описанием меньшем, чем в 100 символов:

i = 0
for d in books["description"]:
    if len(d) < 100:
        books.drop([i], inplace=True)
    i+=1
len(books)

18628

In [8]:
#обновляем индексы:

books = books[~books['description'].isnull()][['genre_and_votes', 'description']]
books['book_id'] = range(len(books))
books.index = range(len(books))

In [9]:
#удаляем пустые строки и строки, содержащие "плохие" (нечитабельные) символы

i = 0
for d in books["description"]:
    if not bool(re.match('^(.[a-zA-Z])', d)):
        books.drop([i], inplace=True)
    i+=1
len(books)

15945

In [10]:
books['genre_and_votes'] = books['genre_and_votes'].str.split(' ').str[0]

books['genre_and_votes'].unique()

array(['Young', 'Christian', 'Economics-Finance', 'Romance', 'Fiction',
       'Sequential', 'Fantasy', 'Mystery', 'Art', 'Nonfiction', 'History',
       'Historical-Historical', 'Horror', 'Fantasy-Paranormal', nan,
       'Romance-Paranormal', 'World', 'Plays', 'Thriller', 'Short',
       'Romance-Romantic', 'Childrens-Picture', 'Classics',
       'Erotica-BDSM', 'Womens', 'Fantasy-Urban', 'New', 'Business',
       'Science', 'Childrens-Middle', 'Paranormal-Vampires', 'Childrens',
       'Cultural-Africa', 'Religion', 'Autobiography-Memoir',
       'Romance-Historical', 'Poetry', 'Dark', 'Novels', 'Philosophy',
       'LGBT', 'Romance-M', 'Economics', 'Politics', 'Humor',
       'Culture-Film', 'Self', 'Food', 'Adult', 'Sports-Sports',
       'Mystery-Crime', 'Media', 'Music', 'Writing-Essays', 'LGBT-Gay',
       'Fairy', 'Fantasy-Magic', 'Horror-Zombies', 'Cultural-Japan',
       'Animals', 'Inspirational', 'Romance-Erotic', 'Paranormal-Witches',
       'Religion-Theology', 'Health-M

In [11]:
#так как длина books изменилась, снова обновляем индексы:

books = books[~books['description'].isnull()][['genre_and_votes', 'description']]
books['book_id'] = range(len(books))
books.index = range(len(books))

In [12]:
# удаляем все неанглоязычные описания (чтобы темы в дальнейшем были угадываемыми)

i = 0
for d in books["description"]:
    #print(i)
    lang = str(detect_langs(d))
    if not bool(re.search("en", lang)):
        #print(d)
        books.drop([i], inplace=True)
    elif int(lang[lang.index('en') + 5]) < 8:
        books.drop([i], inplace=True)        
    i+=1

KeyboardInterrupt: 

In [ ]:
#d='Just a few months shy of her 30th birthday, Gus Curtis finally feels like she has it all: a strong career, great friends, and a wonderful boyfriend. But all of this comes crashing down when Gus discovers Nate, her "Mr. Right," hooking up behind her back with her so-called "friend" Helen. Soon it seems like the life Gus has worked to make so adult looks a lot like the one s Just a few months shy of her 30th birthday, Gus Curtis finally feels like she has it all: a strong career, great friends, and a wonderful boyfriend. But all of this comes crashing down when Gus discovers Nate, her "Mr. Right," hooking up behind her back with her so-called "friend" Helen. Soon it seems like the life Gus has worked to make so adult looks a lot like the one she already had as a teenager, and Gus is left with more questions than answers: Can she win Nate back before she turns 30 alone? (And if so, does she really want him?) Is Helen really as devious and manipulative as she seems, or, worse, is Gus more like her frenemy than she ever imagined? And is she ever going to grow up? With the clock ticking down to her birthday, Gus discovers that sometimes the best thing about best-laid plans is trashing them altogether.'
#lang = str(detect_langs(d))
#print(not bool(re.search("en", str(lang))))
#i = lang.index('en')
#print(int(lang[i+5]))

In [ ]:
len(books["description"])

In [ ]:
#снова обновляем индексы

books = books[~books['description'].isnull()][['genre_and_votes', 'description']]
books['book_id'] = range(len(books))
books.index = range(len(books))

In [ ]:
#меняем название колонки "genre_and_votes", так как в ней теперь только один жанр

books = books.rename(columns={"genre_and_votes": "genre"})
books = books[books['genre'].notna()]
print(len(books))
books.tail(5)

In [ ]:
books["genre"].value_counts()

In [ ]:
#выбираем наиболее популярные жанры:

books = books[books.groupby('genre').genre.transform('count') > 500]

In [ ]:
books[['book_id', 'description']].to_csv("books_descriptions.csv", index=None)

In [ ]:
len(books)

In [ ]:
users = ["u10{}".format(i) for i in range(1000, 8000)]
users = pd.DataFrame({'uid': users, 'churn': [0 for i in range(6000)]+[1 for i in range(1000)]})
users.sample(frac=1).to_csv("users_churn.csv", index=None)
users.tail(3)

In [ ]:
books['genre'].value_counts()

In [ ]:
fiction = books[books['genre']==u'Fiction']['book_id'].values
fantasy = books[books['genre']==u'Fantasy']['book_id'].values
romance = books[books['genre']==u'Romance']['book_id'].values
young = books[books['genre']==u'Young']['book_id'].values
nonfiction = books[books['genre']==u'Nonfiction']['book_id'].values
historical = books[books['genre']==u'Historical-Historical']['book_id'].values
science = books[books['genre']==u'Science']['book_id'].values
mystery = books[books['genre']==u'Mystery']['book_id'].values
#sequential = books[books['genre']==u'Sequential']['book_id'].values
#history = books[books['genre']==u'History']['book_id'].values
#poetry = books[books['genre']==u'Poetry']['book_id'].values
#horror = books[books['genre']==u'Horror']['book_id'].values

In [ ]:
len(mystery)

In [ ]:
total = []
for i in range(2000):
    q = np.random.choice(young, 5).tolist()+np.random.choice(mystery, 1).tolist()
    total.append(q)
for i in range(2000):
    q = np.random.choice(fantasy, 5).tolist()+np.random.choice(young, 1).tolist()
    total.append(q)
for i in range(1000):
    q = np.random.choice(historical, 5).tolist()+np.random.choice(science, 1).tolist()
    total.append(q)
for i in range(1000):
    q = np.random.choice(mystery, 5).tolist()+np.random.choice(historical, 1).tolist()
    total.append(q)
for i in range(1000):
    q = np.random.choice(nonfiction, 5).tolist()+np.random.choice(romance, 1).tolist()
    total.append(q)
#for i in range(1000):
#    q = np.random.choice(sequential, 5).tolist()+np.random.choice(historical, 1).tolist()
#    total.append(q)

In [ ]:
#total

In [ ]:
pd.DataFrame({'uid': ["u10{}".format(i) for i in range(1000, 8000)], 
              'books_descriptions': total}).sample(frac=1).to_csv("users_books.csv", index=None)

In [ ]:
books[books['book_id']==492909]

In [ ]:
pd.DataFrame({'uid': ["u10{}".format(i) for i in range(1000, 8000)], 
              'books_descriptions': total}).sample(frac=1).to_csv("users_books.csv", index=None)

In [ ]:
stopword_en = stopwords.words('english')
print(len(stopword_en))

morph = pymorphy2.MorphAnalyzer()

In [ ]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_en += additional_stopwords
len(stopword_en)

In [ ]:
books = pd.read_csv("books_descriptions.csv")
print(books.shape)
books.head(3)

In [ ]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_en] # [6]
    
    return words_lem_without_stopwords

In [ ]:
%%time
#Запускаем очистку текста. Будет долго...
books['description'] = books['description'].apply(lambda x: clean_text(x), 1)

In [ ]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
books['description'] = books['description'].apply(lambda x: lemmatization(x), 1)

Обучаем модель:

In [ ]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in books['description'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [ ]:
#common_dictionary[10]

Запускаем обучение

In [ ]:
%%time
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

In [ ]:
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [ ]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in books['description'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

In [ ]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

In [ ]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [ ]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in books['description'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['book_id'] = books['book_id'].values
topic_matrix = topic_matrix[['book_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

In [ ]:
#обработка пользователей

In [ ]:
users = pd.read_csv("users_books.csv")
users.head(3)

In [ ]:
doc_dict = dict(zip(topic_matrix['book_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [ ]:
doc_dict[230]

### Функция get_user_embedding, использующая np.mean и результаты модели для неё

In [ ]:
user_articles_list = users['books_descriptions'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

In [ ]:
get_user_embedding(user_articles_list)

In [ ]:
users['books_descriptions'].iloc[33]

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['books_descriptions'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

Обучаем модель

In [ ]:
target = pd.read_csv("users_churn.csv")
target.head(3)

In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

In [ ]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(15)]], 
                                                    X['churn'], random_state=0)

In [ ]:
y_train.unique()

In [ ]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

In [ ]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

#### Precision, Recall, F_score

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

thresholds_mean, fscore_mean, precision_mean, recall_mean = thresholds[ix], fscore[ix], precision[ix], recall[ix]


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#мы уже нашли ранее "оптимальный" порог, когда максимизировали f_score
font = {'size' : 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()

In [ ]:
roc_auc_mean = roc_auc_score(y_test, preds)
print(roc_auc_mean)

### Функция get_user_embedding, использующая np.median и результаты модели для неё

In [ ]:
user_articles_list = users['books_descriptions'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

In [ ]:
get_user_embedding(user_articles_list)

In [ ]:
users['books_descriptions'].iloc[33]

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['books_descriptions'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

Обучаем модель

In [ ]:
target = pd.read_csv("users_churn.csv")
target.head(3)

In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

In [ ]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(15)]], 
                                                    X['churn'], random_state=0)

In [ ]:
y_train.unique()

In [ ]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

In [ ]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

#### Precision, Recall, F_score

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

thresholds_median, fscore_median, precision_median, recall_median = thresholds[ix], fscore[ix], precision[ix], recall[ix]

In [ ]:
#font = {'size' : 15}

#plt.rc('font', **font)

#cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
#plt.figure(figsize=(10, 8))
#plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
#                      title='Confusion matrix')
#plt.savefig("conf_matrix.png")
#plt.show()

#### Roc_auc

In [ ]:
roc_auc_median = roc_auc_score(y_test, preds)
print(roc_auc_median)

### Функция get_user_embedding, использующая np.max и результаты модели для неё

In [ ]:
user_articles_list = users['books_descriptions'].iloc[33]

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [ ]:
get_user_embedding(user_articles_list)

In [ ]:
users['books_descriptions'].iloc[33]

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['books_descriptions'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

Обучаем модель

In [ ]:
target = pd.read_csv("users_churn.csv")
target.head(3)

In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

In [ ]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(15)]], 
                                                    X['churn'], random_state=0)

In [ ]:
y_train.unique()

In [ ]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

In [ ]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

#### Precision, Recall, F_score

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

thresholds_max, fscore_max, precision_max, recall_max = thresholds[ix], fscore[ix], precision[ix], recall[ix]

In [ ]:
#font = {'size' : 15}

#plt.rc('font', **font)

#cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
#plt.figure(figsize=(10, 8))
#plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
#                      title='Confusion matrix')
#plt.savefig("conf_matrix.png")
#plt.show()

#### Roc_auc

In [ ]:
roc_auc_max = roc_auc_score(y_test, preds)
print(roc_auc_max)

### Сравнение работы моделей

In [ ]:
print('Mean_model: Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_mean, 
                                                                                    fscore_mean,
                                                                                    precision_mean,
                                                                                    recall_mean))
print('            Roc_auc=%.3f' % roc_auc_mean)

In [ ]:
print('Median_model: Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_median, 
                                                                                    fscore_median,
                                                                                    precision_median,
                                                                                    recall_median))
print('            Roc_auc=%.3f' % roc_auc_median)

In [ ]:
print('Max_model: Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_max, 
                                                                                    fscore_max,
                                                                                    precision_max,
                                                                                    recall_max))
print('            Roc_auc=%.3f' % roc_auc_max)